In [2]:
# -*- coding: utf-8 -*-
"""
Created on Fri Nov 02 15:51:37 2018

@author: Shawnzy
"""
import warnings
warnings.filterwarnings('ignore')
#%% Imports
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

out = './'

np.random.seed(0)
cancer = pd.read_hdf('../Input_Data/datasets.hdf','cancer')
cancerX = cancer.drop('diagnosis',1).copy().values
cancerY = cancer['diagnosis'].copy().values


madelon = pd.read_hdf('../Input_Data/datasets.hdf','madelon')        
madelonX = madelon.drop('Class',1).copy().values
madelonY = madelon['Class'].copy().values

# Scale Attributes (Very important for clustering)
madelonX = StandardScaler().fit_transform(madelonX) 
cancerX= StandardScaler().fit_transform(cancerX)



In [3]:
import matplotlib.pyplot as plt 

rfc = RandomForestClassifier(n_estimators=100,class_weight='balanced',random_state=5)
fs_madelon = rfc.fit(madelonX,madelonY).feature_importances_ 
tmp = pd.Series(np.sort(fs_madelon)[::-1])
tmp.to_hdf(out+'results.hdf','madelon_rfc',complib='blosc',complevel=9)

fig = plt.figure()
ax = tmp[:50].plot(kind='bar', title="Madelon - RF - Feature Importance vs Dimensions")
ax.set(xlabel="Feature", ylabel="Feature Importance")
plt.savefig(out+'Plots/madelon_rf.png')


fs_cancer = rfc.fit(cancerX,cancerY).feature_importances_ 
tmp = pd.Series(np.sort(fs_cancer)[::-1])
tmp.to_hdf(out+'results.hdf','cancer_rfc',complib='blosc',complevel=9)

fig = plt.figure()
ax = tmp[:30].plot(kind='bar', title="Cancer - RF - Feature Importance vs Dimensions")
ax.set(xlabel="Feature", ylabel="Feature Importance")
plt.savefig(out+'Plots/cancer_rf.png')

In [4]:
dimensions = range(2,400,2)
for dim in dimensions:
    clf = RandomForestClassifier(n_estimators=100,class_weight='balanced',random_state=5)
    clf = clf.fit(madelonX, madelonY)
    model = SelectFromModel(clf, prefit=True, max_features=dim)
    madelonX_new = model.transform(madelonX)
    new_madelon = pd.pandas.DataFrame(np.hstack((madelonX_new,madelonY.reshape(len(madelonX),1))))
    new_madelon.to_hdf(out+'results.hdf','madelon_rf_dim{}'.format(str(dim)),complib='blosc',complevel=9)

dimensions = range(2,28)
for dim in dimensions: 
    clf = RandomForestClassifier(n_estimators=100,class_weight='balanced',random_state=5)
    clf = clf.fit(cancerX, cancerY)
    model = SelectFromModel(clf, prefit=True, max_features=dim)
    cancerX_new = model.transform(cancerX)
    new_cancer = pd.pandas.DataFrame(np.hstack((cancerX_new,cancerY.reshape(len(cancerX),1))))
    new_cancer.to_hdf(out+'results.hdf','cancer_rf_dim{}'.format(str(dim)),complib='blosc',complevel=9)
